In [1]:
import pandas as pd
import numpy as np
import pprint
import itertools
import random
from collections import Counter as ctr
from sodapy import Socrata

In [2]:
client = Socrata('data.cityofnewyork.us',
                  'P5yEsPDgBpkSlfs3G7Kg0DM6o',)

In [3]:
results = client.get("h9gi-nx95", limit=1920000)
results_df = pd.DataFrame.from_records(results)

In [20]:
results_df['crash_date'].replace({'T00:00:00.000':''}, regex=True, inplace=True)
results_df['crash_date_time']=pd.to_datetime(results_df['crash_date'] + results_df['crash_time'], format='%Y-%m-%d%H:%M')

In [21]:
choques_nyc=results_df[results_df['crash_date_time'].dt.year.isin([2019,2020,2021,2022])]

In [22]:
#traduccion de columnas
choques_nyc.rename(columns= {'crash_date_time': 'colision_fecha_hora',
 'crash_date': 'colision_fecha',
 'crash_time': 'colision_hora',
 'on_street_name': 'nombre_de_calle',
 'off_street_name': 'nombre_fuera_de_calle',
 'cross_street_name': 'calle_interseccion',
 'number_of_persons_injured': 'numero_de_personas_heridas',
 'number_of_persons_killed': 'numero_de_personas_fallecidas',
 'number_of_pedestrians_injured': 'numero_de_peatones_heridos',
 'number_of_pedestrians_killed': 'numero_de_peatones_fallecidos',
 'number_of_cyclist_injured': 'numero_de_ciclistas_heridos',
 'number_of_cyclist_killed': 'numero_de_ciclistas_fallecidos',
 'number_of_motorist_injured': 'numero_de_conductores_heridos',
 'number_of_motorist_killed': 'numero_de_conductores_fallecidos',
 'contributing_factor_vehicle_1': 'factor_contribuyente_vehiculo_1',
 'contributing_factor_vehicle_2': 'factor_contribuyente_vehiculo_2',
 'collision_id': 'colision_ID',
 'vehicle_type_code1': 'codigo_tipo_de_vehiculo_1',
 'vehicle_type_code2': 'codigo_tipo_de_vehiculo_2',
 'borough': 'localidad',
 'zip_code': 'codigo_postal',
 'latitude': 'latitud',
 'longitude': 'longitud',
 'location': 'ubicacion',
 'contributing_factor_vehicle_3': 'factor_contribuyente_vehiculo_3',
 'vehicle_type_code_3': 'codigo_tipo_de_vehiculo_3',
 'contributing_factor_vehicle_4': 'factor_contribuyente_vehiculo_4',
 'vehicle_type_code_4': 'codigo_tipo_de_vehiculo_4',
 'contributing_factor_vehicle_5': 'factor_contribuyente_vehiculo_5',
 'vehicle_type_code_5': 'codigo_tipo_de_vehiculo_5',
 'day': 'dia',
 'no_location': 'sin_ubicacion',
 'day_time': 'hora_del_dia'}, inplace= True)
choques_nyc.head(1)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\756886217.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.rename(columns= {'crash_date_time': 'colision_fecha_hora',


,colision_fecha,colision_hora,nombre_de_calle,numero_de_personas_heridas,numero_de_personas_fallecidas,numero_de_peatones_heridos,numero_de_peatones_fallecidos,numero_de_ciclistas_heridos,numero_de_ciclistas_fallecidos,numero_de_conductores_heridos,...,ubicacion,nombre_fuera_de_calle,factor_contribuyente_vehiculo_3,codigo_tipo_de_vehiculo_3,calle_interseccion,factor_contribuyente_vehiculo_4,codigo_tipo_de_vehiculo_4,factor_contribuyente_vehiculo_5,codigo_tipo_de_vehiculo_5,colision_fecha_hora
0,2022-06-29,6:55,THROGS NECK BRIDGE,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-29 06:55:00


In [23]:
choques_nyc.columns

Index(['colision_fecha', 'colision_hora', 'nombre_de_calle',
       'numero_de_personas_heridas', 'numero_de_personas_fallecidas',
       'numero_de_peatones_heridos', 'numero_de_peatones_fallecidos',
       'numero_de_ciclistas_heridos', 'numero_de_ciclistas_fallecidos',
       'numero_de_conductores_heridos', 'numero_de_conductores_fallecidos',
       'factor_contribuyente_vehiculo_1', 'factor_contribuyente_vehiculo_2',
       'colision_ID', 'codigo_tipo_de_vehiculo_1', 'codigo_tipo_de_vehiculo_2',
       'localidad', 'codigo_postal', 'latitud', 'longitud', 'ubicacion',
       'nombre_fuera_de_calle', 'factor_contribuyente_vehiculo_3',
       'codigo_tipo_de_vehiculo_3', 'calle_interseccion',
       'factor_contribuyente_vehiculo_4', 'codigo_tipo_de_vehiculo_4',
       'factor_contribuyente_vehiculo_5', 'codigo_tipo_de_vehiculo_5',
       'colision_fecha_hora'],
      dtype='object')

In [24]:
choques_nyc['anio'] = choques_nyc['colision_fecha_hora'].dt.year
choques_nyc['mes'] = choques_nyc['colision_fecha_hora'].dt.month
choques_nyc['dia'] = choques_nyc['colision_fecha_hora'].dt.day
choques_nyc['hora'] = choques_nyc['colision_fecha_hora'].dt.time

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\2988236535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['anio'] = choques_nyc['colision_fecha_hora'].dt.year
C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\2988236535.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['mes'] = choques_nyc['colision_fecha_hora'].dt.month
C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\2988236535.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [25]:
choques_nyc.columns

Index(['colision_fecha', 'colision_hora', 'nombre_de_calle',
       'numero_de_personas_heridas', 'numero_de_personas_fallecidas',
       'numero_de_peatones_heridos', 'numero_de_peatones_fallecidos',
       'numero_de_ciclistas_heridos', 'numero_de_ciclistas_fallecidos',
       'numero_de_conductores_heridos', 'numero_de_conductores_fallecidos',
       'factor_contribuyente_vehiculo_1', 'factor_contribuyente_vehiculo_2',
       'colision_ID', 'codigo_tipo_de_vehiculo_1', 'codigo_tipo_de_vehiculo_2',
       'localidad', 'codigo_postal', 'latitud', 'longitud', 'ubicacion',
       'nombre_fuera_de_calle', 'factor_contribuyente_vehiculo_3',
       'codigo_tipo_de_vehiculo_3', 'calle_interseccion',
       'factor_contribuyente_vehiculo_4', 'codigo_tipo_de_vehiculo_4',
       'factor_contribuyente_vehiculo_5', 'codigo_tipo_de_vehiculo_5',
       'colision_fecha_hora', 'anio', 'mes', 'dia', 'hora'],
      dtype='object')

In [26]:
#guardo cada columna de codigo de vehiculo en su variable en forma de lista
#y cambio el valor a str por los nan y simbolos especiales
codigo_tipo_de_vehiculo_1 = list(choques_nyc.codigo_tipo_de_vehiculo_1)
codigo_tipo_de_vehiculo_1 = [str(v) for v in codigo_tipo_de_vehiculo_1]
codigo_tipo_de_vehiculo_2 = list(choques_nyc.codigo_tipo_de_vehiculo_2)
codigo_tipo_de_vehiculo_2 = [str(v) for v in codigo_tipo_de_vehiculo_2]
codigo_tipo_de_vehiculo_3 = list(choques_nyc.codigo_tipo_de_vehiculo_3)
codigo_tipo_de_vehiculo_3 = [str(v) for v in codigo_tipo_de_vehiculo_3]
codigo_tipo_de_vehiculo_4 = list(choques_nyc.codigo_tipo_de_vehiculo_4)
codigo_tipo_de_vehiculo_4 = [str(v) for v in codigo_tipo_de_vehiculo_4]
codigo_tipo_de_vehiculo_5 = list(choques_nyc.codigo_tipo_de_vehiculo_5)
codigo_tipo_de_vehiculo_5 = [str(v) for v in codigo_tipo_de_vehiculo_5]

In [27]:
#formateo de primer letra a mayuscula
codigo_tipo_de_vehiculo_1 = [w.title() for w in codigo_tipo_de_vehiculo_1]
codigo_tipo_de_vehiculo_2 = [w.title() for w in codigo_tipo_de_vehiculo_2]
codigo_tipo_de_vehiculo_3 = [w.title() for w in codigo_tipo_de_vehiculo_3]
codigo_tipo_de_vehiculo_4 = [w.title() for w in codigo_tipo_de_vehiculo_4]
codigo_tipo_de_vehiculo_5 = [w.title() for w in codigo_tipo_de_vehiculo_5]

In [28]:
#saque del dataset los modelos de vehiculo mas predominantes
#separando aquellos que tienen apenas apariciones y nada de peso en los numeros
#luego reemplace los nan por unknown
#y aquellos que no forman parte de la lista de los modelos principales
#son reemplazados por uno de esos modelos al azar
#asi me deshago de toda la basura entre los modelos
def normalize_vehicles(abc):
    good_ones = ['Sedan', 'Station Wagon/Sport Utility Vehicle', 'Taxi', 'Pick-Up Truck', 'Box Truck', 'Bike', 'Bus', 'Tractor Truck Diesel', 'Motorcycle', 'Van', 'E-Bike', 'Ambulance', 'E-Scooter', 'Dump', 'Convertible', 'Flat Bed', 'Moped', 'Garbage Or Refuse']
    list_vehicles = [x for x in abc]
    for v in range(0, len(list_vehicles)):
        if list_vehicles[v] == 'Nan':
            list_vehicles[v] = 'Unknown'
        elif list_vehicles[v] not in good_ones:
            list_vehicles[v] = random.choice(good_ones)
    return list_vehicles

In [29]:
#uso la funcion de normalizacion en cada columna de modelo de vehiculo
vehiculo_1 = normalize_vehicles(codigo_tipo_de_vehiculo_1)
vehiculo_2 = normalize_vehicles(codigo_tipo_de_vehiculo_2)
vehiculo_3 = normalize_vehicles(codigo_tipo_de_vehiculo_3)
vehiculo_4 = normalize_vehicles(codigo_tipo_de_vehiculo_4)
vehiculo_5 = normalize_vehicles(codigo_tipo_de_vehiculo_5)

In [30]:
#checkeo que los grupos esten limpios usando counter
vehiculo_3_counter = ctr(vehiculo_3)
vehiculo_3_most_common = vehiculo_3_counter.most_common(50)
print(vehiculo_3_most_common)

[('Unknown', 460565), ('Sedan', 19941), ('Station Wagon/Sport Utility Vehicle', 16423), ('Pick-Up Truck', 996), ('Taxi', 814), ('Box Truck', 448), ('Bus', 277), ('Tractor Truck Diesel', 197), ('Bike', 191), ('Van', 186), ('Motorcycle', 163), ('Convertible', 123), ('Ambulance', 67), ('Flat Bed', 62), ('E-Scooter', 58), ('Dump', 55), ('E-Bike', 55), ('Moped', 44), ('Garbage Or Refuse', 44)]


In [31]:
#aca igual
codigo_tipo_de_vehiculo_3_counter = ctr(codigo_tipo_de_vehiculo_3)
codigo_tipo_de_vehiculo_3_most_common = codigo_tipo_de_vehiculo_3_counter.most_common(50)
print(codigo_tipo_de_vehiculo_3_most_common)

[('Nan', 460565), ('Sedan', 19915), ('Station Wagon/Sport Utility Vehicle', 16399), ('Pick-Up Truck', 975), ('Taxi', 789), ('Box Truck', 423), ('Bus', 248), ('Tractor Truck Diesel', 173), ('Bike', 172), ('Van', 171), ('Motorcycle', 144), ('Convertible', 102), ('Dump', 42), ('Pk', 42), ('Flat Bed', 40), ('E-Bike', 37), ('Ambulance', 37), ('Tow Truck / Wrecker', 36), ('E-Scooter', 34), ('Garbage Or Refuse', 28), ('Moped', 26), ('4 Dr Sedan', 26), ('Tractor Truck Gasoline', 19), ('Carry All', 15), ('Trailer', 15), ('Motorbike', 14), ('Motorscooter', 14), ('Limo', 12), ('Concrete Mixer', 10), ('Chassis Cab', 8), ('Tanker', 7), ('Truck', 6), ('Refrigerated Van', 6), ('School Bus', 6), ('Lift Boom', 5), ('Unk', 5), ('3-Door', 5), ('Unknown', 5), ('Multi-Wheeled Vehicle', 5), ('Tow Truck', 4), ('Flat Rack', 4), ('Bulk Agriculture', 4), ('Stake Or Rack', 4), ('E-Bik', 4), ('Ambul', 4), ('Trail', 4), ('Fdny Fire', 3), ('Garbage Tr', 3), ('Open Body', 3), ('Rv', 3)]


In [32]:
#una vez checkeado que todo esta bien reemplazo las columnas viejas por las limpias
choques_nyc['codigo_tipo_de_vehiculo_1'] = vehiculo_1
choques_nyc['codigo_tipo_de_vehiculo_2'] = vehiculo_2
choques_nyc['codigo_tipo_de_vehiculo_3'] = vehiculo_3
choques_nyc['codigo_tipo_de_vehiculo_4'] = vehiculo_4
choques_nyc['codigo_tipo_de_vehiculo_5'] = vehiculo_5

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\4201526722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['codigo_tipo_de_vehiculo_1'] = vehiculo_1
C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\4201526722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['codigo_tipo_de_vehiculo_2'] = vehiculo_2
C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\4201526722.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [33]:
#replazando nan por Sin especificar en cada columna de factor contribuyente
choques_nyc.factor_contribuyente_vehiculo_1.fillna('Sin Especificar', inplace= True)
choques_nyc.factor_contribuyente_vehiculo_2.fillna('Sin Especificar', inplace= True)
choques_nyc.factor_contribuyente_vehiculo_3.fillna('Sin Especificar', inplace= True)
choques_nyc.factor_contribuyente_vehiculo_4.fillna('Sin Especificar', inplace= True)
choques_nyc.factor_contribuyente_vehiculo_5.fillna('Sin Especificar', inplace= True)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\2178874517.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_1.fillna('Sin Especificar', inplace= True)
C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\2178874517.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_2.fillna('Sin Especificar', inplace= True)
C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\2178874517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [34]:
#traduccion de columna
choques_nyc.factor_contribuyente_vehiculo_1.replace(['Following Too Closely',
'Unspecified',
'Turning Improperly',
'Pavement Slippery',
'Driver Inattention/Distraction',
'Other Vehicular',
'Passing Too Closely',
'Passing or Lane Usage Improper',
'Driver Inexperience',
'Failure to Yield Right-of-Way',
'Brakes Defective',
'Unsafe Speed',
'Backing Unsafely',
'Reaction to Uninvolved Vehicle',
'View Obstructed/Limited',
'Steering Failure',
'Traffic Control Disregarded',
'Drugs (illegal)',
'Aggressive Driving/Road Rage',
'Fell Asleep',
'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
'Alcohol Involvement',
'Unsafe Lane Changing',
'Pavement Defective',
'Other Lighting Defects',
'Oversized Vehicle',
'Animals Action',
'Outside Car Distraction',
'Illnes',
'Driverless/Runaway Vehicle',
'Passenger Distraction',
'Tire Failure/Inadequate',
'Fatigued/Drowsy',
'Glare',
'Cell Phone (hand-Held)',
'Obstruction/Debris',
'Shoulders Defective/Improper',
'Lost Consciousness',
'Accelerator Defective',
'Failure to Keep Right',
'Traffic Control Device Improper/Non-Working',
'Eating or Drinking',
'Vehicle Vandalism',
'Physical Disability',
'Cell Phone (hands-free)',
'Lane Marking Improper/Inadequate',
'Other Electronic Device',
'Using On Board Navigation Device',
'Tow Hitch Defective',
'Texting',
'Headlights Defective',
'Tinted Windows',
'Prescription Medication',
'Listening/Using Headphones',
'Windshield Inadequate'],
['Seguir Muy De Cerca',
'Sin Especificar',
'Giro Inapropiado',
'Pavimento Resbaladizo',
'Conductor Inatento/Distraido',
'Otra Vehicular',
'Pasar Muy De Cerca',
'Pasar O Usar carril Inapropiadamente',
'Inexperiencia Del Conductor',
'No Otorgar Derecho De Paso',
'Frenos Defectuosos',
'Velocidad Peligrosa',
'Reversa Peligrosa',
'Reaccion A Vehiculo No Involucrado',
'Vision Obstruida/Limitada',
'Falla Al Doblar',
'Ignorar Control De Trafico',
'Drogas (ilegales)',
'Conduccion Agresiva',
'Quedarse Dormido',
'Peaton/Ciclista/Otro Error De Peaton/Confusion',
'Alcohol Involucrado',
'Cambio de carril peligroso',
'Pavimento defectuoso',
'Otros Defectos De Iluminacion',
'Vehiculo Demasiado Grande',
'Accion Animal',
'Distraccion De Auto Ajeno',
'Enfermedad',
'Vehiculo Sin Conductor',
'Distraccion De Pasajero',
'Falla/Defecto de neumatico',
'Fatiga/Adormecimiento',
'Encandilamiento',
'Celular(En Mano)',
'Obstruccion/Escombros',
'Guardabarro Defectivo/Inapropiado',
'Perdida De Conciencia',
'Acelerador Defectuoso',
'Fallo De Mantener En Linear Recta',
'Dispositivo De Control De Trafico Roto/Inapropiado',
'Comida O Bebida',
'Vandalismo De Vehiculo',
'Discapacidad Fisica',
'Celular(Manos Libres)',
'Delimitacion de Carrilo Inadecuado',
'Otro Dispositivo Electronico',
'Usar Dispositivo De Navigacion De A Bordo',
'Enganche De Remolque Defectuoso',
'Mensajeria',
'Luces Defectivas',
'Vidrios Polarizados',
'Prescripcion Medica',
'Usar Auriculares',
'Parabrisas Inadecuados'], inplace= True)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\1851416316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_1.replace(['Following Too Closely',


In [35]:
#traduccion de columna
choques_nyc.factor_contribuyente_vehiculo_2.replace(['Following Too Closely',
'Unspecified',
'Turning Improperly',
'Pavement Slippery',
'Driver Inattention/Distraction',
'Other Vehicular',
'Passing Too Closely',
'Passing or Lane Usage Improper',
'Driver Inexperience',
'Failure to Yield Right-of-Way',
'Brakes Defective',
'Unsafe Speed',
'Backing Unsafely',
'Reaction to Uninvolved Vehicle',
'View Obstructed/Limited',
'Steering Failure',
'Traffic Control Disregarded',
'Drugs (illegal)',
'Aggressive Driving/Road Rage',
'Fell Asleep',
'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
'Alcohol Involvement',
'Unsafe Lane Changing',
'Pavement Defective',
'Other Lighting Defects',
'Oversized Vehicle',
'Animals Action',
'Outside Car Distraction',
'Illnes',
'Driverless/Runaway Vehicle',
'Passenger Distraction',
'Tire Failure/Inadequate',
'Fatigued/Drowsy',
'Glare',
'Cell Phone (hand-Held)',
'Obstruction/Debris',
'Shoulders Defective/Improper',
'Lost Consciousness',
'Accelerator Defective',
'Failure to Keep Right',
'Traffic Control Device Improper/Non-Working',
'Eating or Drinking',
'Vehicle Vandalism',
'Physical Disability',
'Cell Phone (hands-free)',
'Lane Marking Improper/Inadequate',
'Other Electronic Device',
'Using On Board Navigation Device',
'Tow Hitch Defective',
'Texting',
'Headlights Defective',
'Tinted Windows',
'Prescription Medication',
'Listening/Using Headphones'],
['Seguir Muy De Cerca',
'Sin Especificar',
'Giro Inapropiado',
'Pavimento Resbaladizo',
'Conductor Inatento/Distraido',
'Otra Vehicular',
'Pasar Muy De Cerca',
'Pasar O Usar carril Inapropiadamente',
'Inexperiencia Del Conductor',
'No Otorgar Derecho De Paso',
'Frenos Defectuosos',
'Velocidad Peligrosa',
'Reversa Peligrosa',
'Reaccion A Vehiculo No Involucrado',
'Vision Obstruida/Limitada',
'Falla Al Doblar',
'Ignorar Control De Trafico',
'Drogas (ilegales)',
'Conduccion Agresiva',
'Quedarse Dormido',
'Peaton/Ciclista/Otro Error De Peaton/Confusion',
'Alcohol Involucrado',
'Cambio de carril peligroso',
'Pavimento defectuoso',
'Otros Defectos De Iluminacion',
'Vehiculo Demasiado Grande',
'Accion Animal',
'Distraccion De Auto Ajeno',
'Enfermedad',
'Vehiculo Sin Conductor',
'Distraccion De Pasajero',
'Falla/Defecto de neumatico',
'Fatiga/Adormecimiento',
'Encandilamiento',
'Celular(En Mano)',
'Obstruccion/Escombros',
'Guardabarro Defectivo/Inapropiado',
'Perdida De Conciencia',
'Acelerador Defectuoso',
'Fallo De Mantener En Linea Recta',
'Dispositivo De Control De Trafico Roto/Inapropiado',
'Comida O Bebida',
'Vandalismo De Vehiculo',
'Discapacidad Fisica',
'Celular(Manos Libres)',
'Delimitacion de Carrilo Inadecuado',
'Otro Dispositivo Electronico',
'Usar Dispositivo De Navigacion De A Bordo',
'Enganche De Remolque Defectuoso',
'Mensajeria',
'Luces Defectivas',
'Vidrios Polarizados',
'Prescripcion Medica',
'Usar Auriculares'], inplace= True)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\3914632134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_2.replace(['Following Too Closely',


In [36]:
#traduccion de columna
choques_nyc.factor_contribuyente_vehiculo_3.replace(['Following Too Closely',
'Unspecified',
'Turning Improperly',
'Pavement Slippery',
'Driver Inattention/Distraction',
'Other Vehicular',
'Passing Too Closely',
'Passing or Lane Usage Improper',
'Driver Inexperience',
'Failure to Yield Right-of-Way',
'Brakes Defective',
'Unsafe Speed',
'Backing Unsafely',
'Reaction to Uninvolved Vehicle',
'View Obstructed/Limited',
'Steering Failure',
'Traffic Control Disregarded',
'Drugs (illegal)',
'Aggressive Driving/Road Rage',
'Fell Asleep',
'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
'Alcohol Involvement',
'Unsafe Lane Changing',
'Pavement Defective',
'Oversized Vehicle',
'Outside Car Distraction',
'Illnes',
'Driverless/Runaway Vehicle',
'Passenger Distraction',
'Tire Failure/Inadequate',
'Glare',
'Obstruction/Debris',
'Lost Consciousness',
'Failure to Keep Right',
'Traffic Control Device Improper/Non-Working',
'Eating or Drinking',
'Cell Phone (hands-free)',
'Other Electronic Device'],
['Seguir Muy De Cerca',
'Sin Especificar',
'Giro Inapropiado',
'Pavimento Resbaladizo',
'Conductor Inatento/Distraido',
'Otra Vehicular',
'Pasar Muy De Cerca',
'Pasar O Usar carril Inapropiadamente',
'Inexperiencia Del Conductor',
'No Otorgar Derecho De Paso',
'Frenos Defectuosos',
'Velocidad Peligrosa',
'Reversa Peligrosa',
'Reaccion A Vehiculo No Involucrado',
'Vision Obstruida/Limitada',
'Falla Al Doblar',
'Ignorar Control De Trafico',
'Drogas (ilegales)',
'Conduccion Agresiva',
'Quedarse Dormido',
'Peaton/Ciclista/Otro Error De Peaton/Confusion',
'Alcohol Involucrado',
'Cambio de carril peligroso',
'Pavimento defectuoso',
'Vehiculo Demasiado Grande',
'Distraccion De Auto Ajeno',
'Enfermedad',
'Vehiculo Sin Conductor',
'Distraccion De Pasajero',
'Falla/Defecto de neumatico',
'Encandilamiento',
'Obstruccion/Escombros',
'Perdida De Conciencia',
'Fallo De Mantener En Linear Recta',
'Dispositivo De Control De Trafico Roto/Inapropiado',
'Comida O Bebida',
'Celular(Manos Libres)',
'Otro Dispositivo Electronico'], inplace= True)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\582211393.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_3.replace(['Following Too Closely',


In [37]:
#traduccion de columna
choques_nyc.factor_contribuyente_vehiculo_4.replace(['Following Too Closely',
'Unspecified',
'Pavement Slippery',
'Driver Inattention/Distraction',
'Other Vehicular',
'Passing Too Closely',
'Passing or Lane Usage Improper',
'Driver Inexperience',
'Failure to Yield Right-of-Way',
'Brakes Defective',
'Unsafe Speed',
'Backing Unsafely',
'Reaction to Uninvolved Vehicle',
'View Obstructed/Limited',
'Traffic Control Disregarded',
'Aggressive Driving/Road Rage',
'Fell Asleep',
'Alcohol Involvement',
'Unsafe Lane Changing',
'Pavement Defective',
'Outside Car Distraction',
'Obstruction/Debris'],
['Seguir Muy De Cerca',
'Sin Especificar',
'Pavimento Resbaladizo',
'Conductor Inatento/Distraido',
'Otra Vehicular',
'Pasar Muy De Cerca',
'Pasar O Usar carril Inapropiadamente',
'Inexperiencia Del Conductor',
'No Otorgar Derecho De Paso',
'Frenos Defectuosos',
'Velocidad Peligrosa',
'Reversa Peligrosa',
'Reaccion A Vehiculo No Involucrado',
'Vision Obstruida/Limitada',
'Ignorar Control De Trafico',
'Conduccion Agresiva',
'Quedarse Dormido',
'Alcohol Involucrado',
'Cambio de carril peligroso',
'Pavimento defectuoso',
'Distraccion De Auto Ajeno',
'Obstruccion/Escombros'], inplace= True)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\3774588508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_4.replace(['Following Too Closely',


In [38]:
#traduccion de columna
choques_nyc.factor_contribuyente_vehiculo_5.replace(['Following Too Closely',
'Unspecified',
'Pavement Slippery',
'Driver Inattention/Distraction',
'Other Vehicular',
'Passing Too Closely',
'Driver Inexperience',
'Failure to Yield Right-of-Way',
'Unsafe Speed',
'Reaction to Uninvolved Vehicle',
'Alcohol Involvement',
'Pavement Defective',
'Outside Car Distraction',
'Obstruction/Debris'],
['Seguir Muy De Cerca',
'Sin Especificar',
'Pavimento Resbaladizo',
'Conductor Inatento/Distraido',
'Otra Vehicular',
'Pasar Muy De Cerca',
'Inexperiencia Del Conductor',
'No Otorgar Derecho De Paso',
'Velocidad Peligrosa',
'Reaccion A Vehiculo No Involucrado',
'Alcohol Involucrado',
'Pavimento defectuoso',
'Distraccion De Auto Ajeno',
'Obstruccion/Escombros'], inplace= True)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\2432672975.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.factor_contribuyente_vehiculo_5.replace(['Following Too Closely',


In [39]:
choques_nyc.factor_contribuyente_vehiculo_1.unique()

array(['Seguir Muy De Cerca', 'Sin Especificar', 'Giro Inapropiado',
       'Pavimento Resbaladizo', 'Conductor Inatento/Distraido',
       'Otra Vehicular', 'Pasar Muy De Cerca',
       'Pasar O Usar carril Inapropiadamente',
       'Inexperiencia Del Conductor', 'No Otorgar Derecho De Paso',
       'Frenos Defectuosos', 'Velocidad Peligrosa', 'Reversa Peligrosa',
       'Reaccion A Vehiculo No Involucrado', 'Vision Obstruida/Limitada',
       'Falla Al Doblar', 'Ignorar Control De Trafico',
       'Drogas (ilegales)', 'Conduccion Agresiva', 'Quedarse Dormido',
       'Peaton/Ciclista/Otro Error De Peaton/Confusion',
       'Alcohol Involucrado', 'Cambio de carril peligroso',
       'Pavimento defectuoso', 'Otros Defectos De Iluminacion',
       'Vehiculo Demasiado Grande', 'Accion Animal',
       'Distraccion De Auto Ajeno', 'Enfermedad',
       'Vehiculo Sin Conductor', 'Distraccion De Pasajero',
       'Falla/Defecto de neumatico', 'Fatiga/Adormecimiento',
       'Encandilamiento',

In [40]:
choques_nyc.factor_contribuyente_vehiculo_2.unique()

array(['Sin Especificar', 'Otra Vehicular', 'Velocidad Peligrosa',
       'Conductor Inatento/Distraido', 'Inexperiencia Del Conductor',
       'No Otorgar Derecho De Paso', 'Seguir Muy De Cerca',
       'Giro Inapropiado', 'Pasar O Usar carril Inapropiadamente',
       'Vision Obstruida/Limitada', 'Vehiculo Demasiado Grande',
       'Pasar Muy De Cerca', 'Cambio de carril peligroso',
       'Ignorar Control De Trafico', 'Pavimento Resbaladizo',
       'Reaccion A Vehiculo No Involucrado',
       'Peaton/Ciclista/Otro Error De Peaton/Confusion',
       'Conduccion Agresiva', 'Reversa Peligrosa', 'Quedarse Dormido',
       'Obstruccion/Escombros', 'Alcohol Involucrado',
       'Distraccion De Auto Ajeno', 'Encandilamiento',
       'Vidrios Polarizados',
       'Dispositivo De Control De Trafico Roto/Inapropiado',
       'Vandalismo De Vehiculo', 'Drogas (ilegales)',
       'Distraccion De Pasajero', 'Falla Al Doblar',
       'Delimitacion de Carrilo Inadecuado', 'Vehiculo Sin Conductor'

In [41]:
choques_nyc.factor_contribuyente_vehiculo_3.unique()

array(['Sin Especificar', 'Pasar O Usar carril Inapropiadamente',
       'Seguir Muy De Cerca', 'Pavimento Resbaladizo', 'Otra Vehicular',
       'Conductor Inatento/Distraido', 'Cambio de carril peligroso',
       'Vision Obstruida/Limitada', 'Distraccion De Auto Ajeno',
       'Reaccion A Vehiculo No Involucrado', 'Ignorar Control De Trafico',
       'No Otorgar Derecho De Paso', 'Vehiculo Demasiado Grande',
       'Obstruccion/Escombros', 'Vehiculo Sin Conductor',
       'Peaton/Ciclista/Otro Error De Peaton/Confusion',
       'Comida O Bebida', 'Velocidad Peligrosa',
       'Inexperiencia Del Conductor', 'Reversa Peligrosa',
       'Conduccion Agresiva', 'Alcohol Involucrado', 'Pasar Muy De Cerca',
       'Dispositivo De Control De Trafico Roto/Inapropiado',
       'Pavimento defectuoso', 'Falla/Defecto de neumatico',
       'Quedarse Dormido', 'Giro Inapropiado',
       'Fallo De Mantener En Linear Recta', 'Celular(Manos Libres)',
       'Otro Dispositivo Electronico', 'Distraccio

In [42]:
choques_nyc.factor_contribuyente_vehiculo_4.unique()

array(['Sin Especificar', 'Conductor Inatento/Distraido',
       'Reaccion A Vehiculo No Involucrado', 'Seguir Muy De Cerca',
       'Otra Vehicular', 'Pavimento Resbaladizo', 'Alcohol Involucrado',
       'Inexperiencia Del Conductor', 'Distraccion De Auto Ajeno',
       'Frenos Defectuosos', 'Pasar O Usar carril Inapropiadamente',
       'Obstruccion/Escombros', 'Pasar Muy De Cerca',
       'Ignorar Control De Trafico', 'Vision Obstruida/Limitada',
       'Quedarse Dormido', 'No Otorgar Derecho De Paso',
       'Velocidad Peligrosa', 'Reversa Peligrosa', 'Conduccion Agresiva',
       'Cambio de carril peligroso', 'Pavimento defectuoso'], dtype=object)

In [43]:
choques_nyc.factor_contribuyente_vehiculo_5.unique()

array(['Sin Especificar', 'Reaccion A Vehiculo No Involucrado',
       'Otra Vehicular', 'Seguir Muy De Cerca',
       'Conductor Inatento/Distraido', 'Alcohol Involucrado',
       'Pasar Muy De Cerca', 'Distraccion De Auto Ajeno',
       'Inexperiencia Del Conductor', 'Pavimento Resbaladizo',
       'Obstruccion/Escombros', 'Velocidad Peligrosa',
       'No Otorgar Derecho De Paso', 'Pavimento defectuoso'], dtype=object)

In [44]:
choques_nyc.factor_contribuyente_vehiculo_5.unique()

array(['Sin Especificar', 'Reaccion A Vehiculo No Involucrado',
       'Otra Vehicular', 'Seguir Muy De Cerca',
       'Conductor Inatento/Distraido', 'Alcohol Involucrado',
       'Pasar Muy De Cerca', 'Distraccion De Auto Ajeno',
       'Inexperiencia Del Conductor', 'Pavimento Resbaladizo',
       'Obstruccion/Escombros', 'Velocidad Peligrosa',
       'No Otorgar Derecho De Paso', 'Pavimento defectuoso'], dtype=object)

In [45]:
choques_nyc.codigo_tipo_de_vehiculo_1.unique()

array(['Sedan', 'Taxi', 'Station Wagon/Sport Utility Vehicle', 'Bike',
       'Pick-Up Truck', 'Box Truck', 'Bus', 'Unknown', 'Ambulance',
       'E-Bike', 'Van', 'Flat Bed', 'Moped', 'Tractor Truck Diesel',
       'Motorcycle', 'Garbage Or Refuse', 'Convertible', 'Dump',
       'E-Scooter'], dtype=object)

In [46]:
choques_nyc.codigo_tipo_de_vehiculo_1.replace(
    ['Station Wagon/Sport Utility Vehicle',
    'Pick-Up Truck',
    'Box Truck',
    'Bus',
    'Unknown',
    'Ambulance',
    'Bike',
    'Van',
    'Flat Bed',
    'Moped',
    'Tractor Truck Diesel',
    'Dump',
    'Motorcycle',
    'Garbage Or Refuse'],
    ['Camioneta/Vehiculo Utilitario Deportivo',
    'Camioneta',
    'Camion De Caja',
    'Autobus',
    'Desconocido',
    'Ambulancia',
    'Motocicleta',
    'Furgoneta',
    'Camion',
    'Motocicleta',
    'Tractor',
    'Camion De Basura',
    'Motocicleta',
    'Camion De Basura'], inplace= True
)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\206431653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.codigo_tipo_de_vehiculo_1.replace(


In [47]:
choques_nyc.codigo_tipo_de_vehiculo_2.replace(
    ['Station Wagon/Sport Utility Vehicle',
    'Pick-Up Truck',
    'Box Truck',
    'Bus',
    'Unknown',
    'Ambulance',
    'Bike',
    'Van',
    'Flat Bed',
    'Moped',
    'Tractor Truck Diesel',
    'Dump',
    'Motorcycle',
    'Garbage Or Refuse'],
    ['Camioneta/Vehiculo Utilitario Deportivo',
    'Camioneta',
    'Camion De Caja',
    'Autobus',
    'Desconocido',
    'Ambulancia',
    'Motocicleta',
    'Furgoneta',
    'Camion',
    'Motocicleta',
    'Tractor',
    'Camion De Basura',
    'Motocicleta',
    'Camion De Basura'], inplace= True
)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\51666981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.codigo_tipo_de_vehiculo_2.replace(


In [48]:
choques_nyc.codigo_tipo_de_vehiculo_3.replace(
    ['Station Wagon/Sport Utility Vehicle',
    'Pick-Up Truck',
    'Box Truck',
    'Bus',
    'Unknown',
    'Ambulance',
    'Bike',
    'Van',
    'Flat Bed',
    'Moped',
    'Tractor Truck Diesel',
    'Dump',
    'Motorcycle',
    'Garbage Or Refuse'],
    ['Camioneta/Vehiculo Utilitario Deportivo',
    'Camioneta',
    'Camion De Caja',
    'Autobus',
    'Desconocido',
    'Ambulancia',
    'Motocicleta',
    'Furgoneta',
    'Camion',
    'Motocicleta',
    'Tractor',
    'Camion De Basura',
    'Motocicleta',
    'Camion De Basura'], inplace= True
)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\830710450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.codigo_tipo_de_vehiculo_3.replace(


In [49]:
choques_nyc.codigo_tipo_de_vehiculo_4.replace(
    ['Station Wagon/Sport Utility Vehicle',
    'Pick-Up Truck',
    'Box Truck',
    'Bus',
    'Unknown',
    'Ambulance',
    'Bike',
    'Van',
    'Flat Bed',
    'Moped',
    'Tractor Truck Diesel',
    'Dump',
    'Motorcycle',
    'Garbage Or Refuse'],
    ['Camioneta/Vehiculo Utilitario Deportivo',
    'Camioneta',
    'Camion De Caja',
    'Autobus',
    'Desconocido',
    'Ambulancia',
    'Motocicleta',
    'Furgoneta',
    'Camion',
    'Motocicleta',
    'Tractor',
    'Camion De Basura',
    'Motocicleta',
    'Camion De Basura'], inplace= True
)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\439636689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.codigo_tipo_de_vehiculo_4.replace(


In [50]:
choques_nyc.codigo_tipo_de_vehiculo_5.replace(
    ['Station Wagon/Sport Utility Vehicle',
    'Pick-Up Truck',
    'Box Truck',
    'Bus',
    'Unknown',
    'Ambulance',
    'Bike',
    'Van',
    'Flat Bed',
    'Moped',
    'Tractor Truck Diesel',
    'Dump',
    'Motorcycle',
    'Garbage Or Refuse'],
    ['Camioneta/Vehiculo Utilitario Deportivo',
    'Camioneta',
    'Camion De Caja',
    'Autobus',
    'Desconocido',
    'Ambulancia',
    'Motocicleta',
    'Furgoneta',
    'Camion',
    'Motocicleta',
    'Tractor',
    'Camion De Basura',
    'Motocicleta',
    'Camion De Basura'], inplace= True
)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\1373640272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc.codigo_tipo_de_vehiculo_5.replace(


In [51]:
choques_nyc.codigo_tipo_de_vehiculo_1.unique()

array(['Sedan', 'Taxi', 'Camioneta/Vehiculo Utilitario Deportivo',
       'Motocicleta', 'Camioneta', 'Camion De Caja', 'Autobus',
       'Desconocido', 'Ambulancia', 'E-Bike', 'Furgoneta', 'Camion',
       'Tractor', 'Camion De Basura', 'Convertible', 'E-Scooter'],
      dtype=object)

In [52]:
choques_nyc.columns

Index(['colision_fecha', 'colision_hora', 'nombre_de_calle',
       'numero_de_personas_heridas', 'numero_de_personas_fallecidas',
       'numero_de_peatones_heridos', 'numero_de_peatones_fallecidos',
       'numero_de_ciclistas_heridos', 'numero_de_ciclistas_fallecidos',
       'numero_de_conductores_heridos', 'numero_de_conductores_fallecidos',
       'factor_contribuyente_vehiculo_1', 'factor_contribuyente_vehiculo_2',
       'colision_ID', 'codigo_tipo_de_vehiculo_1', 'codigo_tipo_de_vehiculo_2',
       'localidad', 'codigo_postal', 'latitud', 'longitud', 'ubicacion',
       'nombre_fuera_de_calle', 'factor_contribuyente_vehiculo_3',
       'codigo_tipo_de_vehiculo_3', 'calle_interseccion',
       'factor_contribuyente_vehiculo_4', 'codigo_tipo_de_vehiculo_4',
       'factor_contribuyente_vehiculo_5', 'codigo_tipo_de_vehiculo_5',
       'colision_fecha_hora', 'anio', 'mes', 'dia', 'hora'],
      dtype='object')

In [53]:
choques_nyc['tipo_causante']= choques_nyc['codigo_tipo_de_vehiculo_1']

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\2697252169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['tipo_causante']= choques_nyc['codigo_tipo_de_vehiculo_1']


In [62]:
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_1 != 'Desconocido'),'codigo_tipo_de_vehiculo_1']=1
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_2 != 'Desconocido'),'codigo_tipo_de_vehiculo_2']=1
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_3 != 'Desconocido'),'codigo_tipo_de_vehiculo_3']=1
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_4 != 'Desconocido'),'codigo_tipo_de_vehiculo_4']=1
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_5 != 'Desconocido'),'codigo_tipo_de_vehiculo_5']=1

In [63]:
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_1 == 'Desconocido'),'codigo_tipo_de_vehiculo_1']=0
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_2 == 'Desconocido'),'codigo_tipo_de_vehiculo_2']=0
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_3 == 'Desconocido'),'codigo_tipo_de_vehiculo_3']=0
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_4 == 'Desconocido'),'codigo_tipo_de_vehiculo_4']=0
choques_nyc.loc[(choques_nyc.codigo_tipo_de_vehiculo_5 == 'Desconocido'),'codigo_tipo_de_vehiculo_5']=0

In [65]:
choques_nyc['cantidad_autos_choque']= choques_nyc['codigo_tipo_de_vehiculo_1']+choques_nyc['codigo_tipo_de_vehiculo_2'] + choques_nyc['codigo_tipo_de_vehiculo_3'] + choques_nyc['codigo_tipo_de_vehiculo_4'] + choques_nyc['codigo_tipo_de_vehiculo_5']

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\2346095221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['cantidad_autos_choque']= choques_nyc['codigo_tipo_de_vehiculo_1']+choques_nyc['codigo_tipo_de_vehiculo_2'] + choques_nyc['codigo_tipo_de_vehiculo_3'] + choques_nyc['codigo_tipo_de_vehiculo_4'] + choques_nyc['codigo_tipo_de_vehiculo_5']


In [67]:
choques_nyc['numero_de_personas_heridas'].fillna(0, inplace=True)
choques_nyc['numero_de_personas_fallecidas'].fillna(0, inplace=True)
choques_nyc['numero_de_peatones_heridos'].fillna(0, inplace=True)
choques_nyc['numero_de_peatones_fallecidos'].fillna(0, inplace=True)
choques_nyc['numero_de_ciclistas_heridos'].fillna(0, inplace=True)
choques_nyc['numero_de_ciclistas_fallecidos'].fillna(0, inplace=True)
choques_nyc['numero_de_conductores_heridos'].fillna(0, inplace=True)
choques_nyc['numero_de_conductores_fallecidos'].fillna(0, inplace=True)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\1774733682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['numero_de_personas_heridas'].fillna(0, inplace=True)
C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\1774733682.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['numero_de_personas_fallecidas'].fillna(0, inplace=True)
C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\1774733682.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [69]:
choques_nyc['numero_de_personas_heridas']=choques_nyc['numero_de_personas_heridas'].astype(int)
choques_nyc['numero_de_personas_fallecidas']= choques_nyc['numero_de_personas_fallecidas'].astype(int)
choques_nyc['numero_de_peatones_heridos']= choques_nyc['numero_de_peatones_heridos'].astype(int)
choques_nyc['numero_de_peatones_fallecidos']= choques_nyc['numero_de_peatones_fallecidos'].astype(int)
choques_nyc['numero_de_ciclistas_heridos']= choques_nyc['numero_de_ciclistas_heridos'].astype(int)
choques_nyc['numero_de_ciclistas_fallecidos']= choques_nyc['numero_de_ciclistas_fallecidos'].astype(int)
choques_nyc['numero_de_conductores_heridos']= choques_nyc['numero_de_conductores_heridos'].astype(int)
choques_nyc['numero_de_conductores_fallecidos']= choques_nyc['numero_de_conductores_fallecidos'].astype(int)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\3398349128.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['numero_de_personas_heridas']=choques_nyc['numero_de_personas_heridas'].astype(int)
C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\3398349128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['numero_de_personas_fallecidas']= choques_nyc['numero_de_personas_fallecidas'].astype(int)
C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\3398349128.py:3: SettingWithCopyWarni

In [71]:
choques_nyc['numero_de_personas_fallecidas'] = choques_nyc['numero_de_personas_fallecidas'].apply(lambda x: x*2)
choques_nyc['numero_de_peatones_fallecidos'] = choques_nyc['numero_de_peatones_fallecidos'].apply(lambda x: x*2)
choques_nyc['numero_de_ciclistas_fallecidos'] = choques_nyc['numero_de_ciclistas_fallecidos'].apply(lambda x: x*2)
choques_nyc['numero_de_conductores_fallecidos'] = choques_nyc['numero_de_conductores_fallecidos'].apply(lambda x: x*2)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\146870644.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['numero_de_personas_fallecidas'] = choques_nyc['numero_de_personas_fallecidas'].apply(lambda x: x*2)
C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\146870644.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['numero_de_peatones_fallecidos'] = choques_nyc['numero_de_peatones_fallecidos'].apply(lambda x: x*2)
C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\146870644.py

In [73]:
columnas=['numero_de_personas_fallecidas', 'numero_de_peatones_fallecidos', 'numero_de_ciclistas_fallecidos', 'numero_de_conductores_fallecidos', 'numero_de_personas_heridas', 'numero_de_peatones_heridos', 'numero_de_ciclistas_heridos', 'numero_de_conductores_heridos']

In [74]:
choques_nyc['letalidad']= choques_nyc[columnas].sum(axis=1)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\467996619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['letalidad']= choques_nyc[columnas].sum(axis=1)


In [76]:
choques_nyc.numero_de_conductores_fallecidos.unique()

array([0, 2, 6, 4, 8], dtype=int64)

In [78]:
choques_nyc['latitud']= choques_nyc['latitud'].astype(float)
choques_nyc['longitud']= choques_nyc['longitud'].astype(float)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\244482025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['latitud']= choques_nyc['latitud'].astype(float)
C:\Users\Kevin\AppData\Local\Temp\ipykernel_19020\244482025.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choques_nyc['longitud']= choques_nyc['longitud'].astype(float)


In [79]:
choques_nyc=choques_nyc.dropna(subset=['latitud', 'longitud']).reset_index(drop=True)

In [81]:
prototipo= pd.DataFrame().assign(id_choque=choques_nyc['colision_ID'],fecha_y_hora= choques_nyc['colision_fecha_hora'],anio= choques_nyc['anio'],mes=choques_nyc['mes'],dia=choques_nyc['dia'],latitud=choques_nyc['latitud'], longitud=choques_nyc['longitud'], auto_causante=choques_nyc['tipo_causante'],localidad=choques_nyc['localidad'],codigo_potal=choques_nyc['codigo_postal'], cantidad_autos_choque=choques_nyc['cantidad_autos_choque'],letalidad=choques_nyc['letalidad'] )

In [82]:
prototipo = prototipo.set_index('id_choque')

In [83]:
principal_sql=prototipo 
principal_sql.drop('fecha_y_hora', axis=1, inplace=True)

In [84]:
prototipo.columns

Index(['anio', 'mes', 'dia', 'latitud', 'longitud', 'auto_causante',
       'localidad', 'codigo_potal', 'cantidad_autos_choque', 'letalidad'],
      dtype='object')

In [85]:
principal_sql.columns

Index(['anio', 'mes', 'dia', 'latitud', 'longitud', 'auto_causante',
       'localidad', 'codigo_potal', 'cantidad_autos_choque', 'letalidad'],
      dtype='object')

In [86]:
prototipo.auto_causante.unique()

array(['Taxi', 'Sedan', 'Camioneta/Vehiculo Utilitario Deportivo',
       'Motocicleta', 'Camioneta', 'Camion De Caja', 'Desconocido',
       'Ambulancia', 'E-Bike', 'Furgoneta', 'Camion', 'Tractor',
       'Autobus', 'Camion De Basura', 'Convertible', 'E-Scooter'],
      dtype=object)

In [87]:
principal_sql.auto_causante.unique()

array(['Taxi', 'Sedan', 'Camioneta/Vehiculo Utilitario Deportivo',
       'Motocicleta', 'Camioneta', 'Camion De Caja', 'Desconocido',
       'Ambulancia', 'E-Bike', 'Furgoneta', 'Camion', 'Tractor',
       'Autobus', 'Camion De Basura', 'Convertible', 'E-Scooter'],
      dtype=object)

In [ ]:
prototipo['auto_causante']=prototipo['auto_causante'].str.replace('[^\w\s]', '')

In [236]:
prototipo.to_csv('principal_sql.csv', sep='|')

In [88]:
prototipo.head(10)

,anio,mes,dia,latitud,longitud,auto_causante,localidad,codigo_potal,cantidad_autos_choque,letalidad
id_choque,,,,,,,,,,
4542062,2022,6,30,40.669792,-73.892400,Taxi,BROOKLYN,11207,1,2
4407147,2021,4,13,40.683580,-73.976170,Sedan,BROOKLYN,11217,1,2
4542376,2022,6,30,40.850475,-73.915436,Sedan,BRONX,10453,2,0
4406885,2021,4,12,0.000000,0.000000,Camioneta/Vehiculo Utilitario Deportivo,NaN,NaN,2,0
4136992,2019,5,21,40.697540,-73.983120,Motocicleta,BROOKLYN,11201,2,0
4395664,2021,2,26,40.843464,-73.836000,Camioneta/Vehiculo Utilitario Deportivo,BRONX,10461,1,0
4397513,2021,3,9,40.692547,-73.990974,Camioneta,NaN,NaN,2,2
4403773,2021,3,31,40.626457,-73.918000,Sedan,BROOKLYN,11234,2,2
4405244,2021,4,6,40.526894,-74.167280,Camioneta/Vehiculo Utilitario Deportivo,STATEN ISLAND,10312,2,14
